# Topics


In [65]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [5]:
with open("06-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'..#....##....................#...#...#..........................................................................................#.\n.....#..............................................#....#.....#.....#..............................#.......#...#.............#...\n............#.#.........................................#...................#..#...........................###..........#.....#...\n.....#.....#.............#...........................................................#.#.................#.........#..............\n.......................#...........................#................#...........#......#.....................#...#.......#....#...\n...#........................................#....................#.......#........##.##.........#.........................#.......\n......#...........#....................#...........#.............................#........................#....................#..\n...........................#...................................#...........

In [20]:
test_data_raw = r"""....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""



def preprocess_data (data):
    # dtype='U10'
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')


test_data = preprocess_data(test_data_raw)
# test_data2 = preprocess_data(test_data_raw2)
display(test_data)

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U10')

In [21]:
data = preprocess_data(data_raw)
data

array([['.', '.', '#', ..., '.', '#', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U10')

In [28]:
def solution (data, verbose=False):

    start_x, start_y = np.argwhere(np.isin(data, ["^", ">", "<", "v"]))[0].tolist()
    
    next_dir = {"^": ">", ">": "v", "v": "<", "<":"^"}
    step_dir = {"^": (-1,0), ">": (0,1), "v": (1,0), "<": (0,-1)}
    
    def next_position(field, x,y):
        done = False
        direction = field[x,y]
        next_x = x + step_dir[direction][0]
        next_y = y + step_dir[direction][1]

        
        if next_x < 0 or next_x >= field.shape[0] or next_y < 0 or next_y >= field.shape[1]:
            done = True
        elif field[next_x, next_y] == "#":
            direction = next_dir[direction]
            next_x = x + step_dir[direction][0]
            next_y = y + step_dir[direction][1]
            
        return done, next_x, next_y, direction
    
    running = True
    field = data.copy()
    steps = 1
    while running:
        done, next_x, next_y, direction = next_position(field, start_x, start_y)
        if done:
            running = False
            break
        else:
            field[start_x, start_y] = "X"
            if field[next_x, next_y] == ".":
                steps += 1
            field[next_x, next_y] = direction
            start_x = next_x
            start_y = next_y
            
    
    return steps




sol = solution(test_data, verbose=True)

print(sol)


41


In [29]:
sol = solution(data)
print(sol)

5095


# Part 2

In [76]:
def solution2 (data, verbose=False):

    first_x, first_y = np.argwhere(np.isin(data, ["^", ">", "<", "v"]))[0].tolist()
    
    next_dir = {"^": ">", ">": "v", "v": "<", "<":"^"}
    step_dir = {"^": (-1,0), ">": (0,1), "v": (1,0), "<": (0,-1)}
    
    def next_position(field, x,y):
        done = False
        direction = field[x,y]
        next_x = x + step_dir[direction][0]
        next_y = y + step_dir[direction][1]
        rotated = False
        
        if next_x < 0 or next_x >= field.shape[0] or next_y < 0 or next_y >= field.shape[1]:
            done = True
        elif field[next_x, next_y] == "#" or field[next_x, next_y] == "O":
            rotated = True
            
            direction = next_dir[direction]
            next_x = x + step_dir[direction][0]
            next_y = y + step_dir[direction][1]
            
            # Rotate again if another obstacle is encountered
            if field[next_x, next_y] == "#" or field[next_x, next_y] == "O":
                direction = next_dir[direction]
                next_x = x + step_dir[direction][0]
                next_y = y + step_dir[direction][1]
            
        return done, next_x, next_y, direction, rotated
    
    def trace_field (data, x,y):
        obstructions = []
        
        running = True
        field = data.copy()
        start_x, start_y = x,y
        
        while running:
            done, next_x, next_y, direction, rotated = next_position(field, start_x, start_y)
            if done:
                running = False
                break
            else:
                if (next_x != x or next_y != y ):
                    obstructions.append((next_x, next_y))
                
                field[start_x, start_y] = direction
                field[next_x, next_y] = direction
                start_x = next_x
                start_y = next_y
        

        return obstructions
           
    def loop_check (field, x, y, obstruct):
        running = True
        field[obstruct] = 'O'
        start_x, start_y = x,y
        looping = False
        
        visited = set()
        
        # print(field)
        while running:
            done, next_x, next_y, direction, rotated = next_position(field, start_x, start_y)
            if done:
                running = False
                break
            elif (next_x, next_y, direction) in visited:
                looping = True
                # print(f"Loops at {(next_x, next_y)}", (next_x, next_y) )
                break
            else:
                visited.add((next_x, next_y, direction))
                field[start_x, start_y] = direction
                start_x = next_x
                start_y = next_y
                field[next_x, next_y] = direction
        return looping
            
    obstructions = trace_field(data, first_x, first_y)
    # print(obstructions,"\n" , trace)
    
    loops = 0
    for obstruction in tqdm(set(obstructions)):
        looped = loop_check(data.copy(), first_x, first_y, obstruction )
        if looped:
            loops += 1

    return loops





sol = solution2(test_data, verbose=True)
print(sol)
# print(dists)

# display(sum(sol))

    

100%|██████████| 40/40 [00:00<00:00, 9996.55it/s]

6


In [77]:
sol = solution2(data)
print(sol)

  0%|          | 0/5094 [00:00<?, ?it/s]

100%|██████████| 5094/5094 [00:50<00:00, 101.61it/s]

1933
